In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import spacy
from spacy.language import Language
from spacy_language_detection import LanguageDetector
from Levenshtein import distance

In [2]:
def read_file(path):
    with open(path, "r", encoding="utf-8") as f:
        lines = [line.strip() for line in f.readlines()]
    return lines

def save_file(file, path):
    with open(path, "w") as f:
        for line in file:
            f.write(line + "\n")

In [3]:
de = read_file("data/raw/train.de")
# en = read_file("data/raw/train.en")

In [4]:
amp_entities = {
    '&quot;': '',
    '&apos;': "'",
    '&amp;': 'und',
    '&lt;': '',
    '&gt;': '',
    '&#124;': '',
    '&#93;': ')',
    '&#91;': '(',
}

In [5]:
def remove_quotes(string):
    # Replace „“” with "
    string = string.replace('„', '"').replace('“', '"').replace('”', '"').replace('’', '"').replace('‘', '"')
    # Remove " if not closed, not ideal way but it should be enough
    if '"' in string and string.count('"') % 2 != 0:
        string = string.replace('"', '')
    return string


def clean_de_strings(strings):
    transformed_strings = []
    
    for string in tqdm(strings, desc="Cleaning strings", total=len(strings), ncols=100):

        # Replace amp_entities
        for entity, replacement in amp_entities.items():
            string = string.replace(entity, replacement)
        
        # Replace {} and [] with ()
        string = string.replace('{', '(').replace('}', ')')
        string = string.replace('[', '(').replace(']', ')')

        # Replace weird symbols
        string = string.replace('Гј', 'ü')
        string = string.replace('Г ¶ ', 'ö')
        string = string.replace('Г ¤ ', 'ä')
        string = string.replace('Гџ', 'ß')
        string = string.replace('–', '-')
        string = string.replace('� ber', 'über')

        # Delete weird symbols
        string = re.sub(r'##UNDERSCORE##', '', string)
        string = re.sub(r'##AT##-##AT##', '', string)
        string = re.sub(r'##AT##', '', string)
        string = re.sub(r'##STAR##', '', string)
        string = re.sub(r'\(\s*\)', '', string)
        string = re.sub(r'\( . \)', '', string)
        string = re.sub(r'[-]+', '', string)
        string = re.sub(r'/\s*/', '', string)
        string = re.sub(r'\\', '', string)
        string = re.sub(r'[<>®™©°«»…†„“”Σ‡‰‹•¦+=±¿]', '', string)
        string = re.sub(r'\( \(', '', string)
        string = re.sub(r'\) \)', '', string)

        # Replace excessive characters
        string = re.sub(r'\.(\s*\.)+', '.', string)
        string = re.sub(r',(\s*,)+', ',', string)
        string = re.sub(r'\!(\s*\!)+', '!', string)
        string = re.sub(r'\?(\s*\?)+', '?', string)
        string = re.sub(r':(\s*:)+', ':', string)
        string = re.sub(r';(\s*;)+', ';', string)

        # Remove quotes
        string = remove_quotes(string)

        # # Remove excessive punctuation
        string = re.sub(r'([,:\.;]\s){2,}', r'', string)
        string = re.sub(r',\s*\.$', '.', string)
        string = re.sub(r', \. ([A-Z])', '\. \1', string)
        string = re.sub(r', \. ([a-z])', ', \1', string)
        
        # Remove excessive spaces
        string = " ".join(string.split())

        # Lowercase string
        string = string.lower()

        # Add cleaned string to list
        transformed_strings.append(string)
    return transformed_strings

In [6]:
clean_de_strings(["ax : 41 ( 91 ) 910 57 77 ) , . checkin : 14 : 00 ; checkout ", "als � bereinander , . Der BenQ ( . ) ( 3 ) , . , ; Bordeaux , trocken rot + • = und quot gereicht , .", "ist grГ ¶ sser Jahres mГ ¶ glich . ausschlieГџlich KanГ ¤ len der Ukraine  Theaterfestival РЃв – СЊС ‘ СЉС € С ‰ die Arche .", 'Wendn„“” ##AT##-##AT##... /  / u . {hello} .. ( ) keine Seiten / mit Schlusselwortern oder Unterkategorien entdeckst , die sich auf die Texte in deiner Homepage beziehen , kannst du deine eigenen Seiten mit Schlusselwortern oder Unterkategorien vorschlagen und schaffen , / i > verbunden mit … " . satellite .  und du kannst deine kontextuellen Listings dort unentgeltlich und in realer Zeit hinzufugen .', "Diese ( ) Funktion ()ist ein  Alias für : imap ##UNDERSCORE## listscan ( ) .", "Diese ! Funktion ist ein ! ! ! Alias für ! !! : OCI  --Collection--- > free .", "Diese Funktion ist ? ? ? ein Alias ?? für : imap ##UNDERSCORE## listscan ( ) .", "Ferienwohnungen | Hotels | Pensionen | Campings | Unterhaltung | Last Minute Angebote ! !", 'Was heißt hier eigentlich " auf eigene Gefahr  ?', 'Es ist nicht so gravierend wie das , worüber Sie hier diskutieren , aber in allen Mitgliedstaaten gibt es gelegentlich Naturkatastrophen und ich denke dabei auch an die Countys , die ich vertrete - Offaly , http : / / {} en.wikipedia.org / wiki / County ##UNDERSCORE## Laois " \ o " County Laois  und Louth - wo es außerhalb der üblichen Saison äußerst ungewöhnliche Überschwemmungen gab .', "( Beispiel ( ( Eris ) ) ) ( ( ( DS9 Der Plan des Dominion ) ) ) Der ( ( Glaube ) ) der"])

Cleaning strings: 100%|███████████████████████████████████████████| 11/11 [00:00<00:00, 5576.18it/s]


['ax : 41 ( 91 ) 910 57 77 ) checkin : 14 : 00 ; checkout',
 'als übereinander der benq bordeaux , trocken rot und quot gereicht .',
 'ist grösser jahres möglich . ausschließlich kanälen der ukraine theaterfestival рѓв сњс сљс € с die arche .',
 'wendn . u . (hello) . keine seiten / mit schlusselwortern oder unterkategorien entdeckst , die sich auf die texte in deiner homepage beziehen , kannst du deine eigenen seiten mit schlusselwortern oder unterkategorien vorschlagen und schaffen , / i verbunden mit . satellite . und du kannst deine kontextuellen listings dort unentgeltlich und in realer zeit hinzufugen .',
 'diese funktion ist ein alias für : imap listscan .',
 'diese ! funktion ist ein ! alias für ! : oci collection free .',
 'diese funktion ist ? ein alias ? für : imap listscan .',
 'ferienwohnungen | hotels | pensionen | campings | unterhaltung | last minute angebote !',
 'was heißt hier eigentlich auf eigene gefahr ?',
 'es ist nicht so gravierend wie das , worüber sie hier di

In [7]:
de_clean = clean_de_strings(de)
save_file(de_clean, "data/clean/clean.de")
de_clean = read_file("data/clean/clean.de")
save_file(de_clean, "data/clean/clean.de")

Cleaning strings: 100%|████████████████████████████████| 4468840/4468840 [02:35<00:00, 28794.71it/s]


In [8]:
de_clean = read_file("data/clean/clean.de")

In [9]:
def extract_words_with_separator(strings, separator="�"):
    pattern = r'[a-zA-Z]+\s+' + re.escape(separator) + r'\s+[a-zA-Z]+'
    extracted_words = []

    for string in tqdm(strings, desc="Extracting words", total=len(strings), ncols=100):
        matches = re.findall(pattern, string)
        for match in matches:
            match = match.replace(" ", "").replace(separator, "_").lower()
            extracted_words.append(match)

    return set(extracted_words)

words = extract_words_with_separator(de_clean)
len(words)

Extracting words: 100%|████████████████████████████████| 4468840/4468840 [01:15<00:00, 59444.86it/s]


7027

In [9]:
def get_lang_detector(nlp, name):
    return LanguageDetector(seed=42)

nlp_model = spacy.load("de_core_news_sm")
Language.factory("language_detector", func=get_lang_detector)
nlp_model.add_pipe('language_detector', last=True)

In [11]:
def resolve_unk_words(words, nlp_model):
    special_characters = list("äöü")
    resolved_words = []
    unresolved_words = []
    for word in tqdm(words, desc="Resolving words", total=len(words), ncols=100):
        ok = False
        for character in special_characters:
            option = word.replace("_", character)
            doc = nlp_model(option)
            language = doc._.language
            if language["language"] == "de" and language["score"] > 0.9:
                resolved_words.append(option)
                ok = True
                break
        if not ok:
            unresolved_words.append(word)
    return resolved_words, unresolved_words

In [12]:
resolved, unresolved = resolve_unk_words(words, nlp_model)
len(resolved), len(unresolved)

Resolving words: 100%|██████████████████████████████████████████| 7027/7027 [02:39<00:00, 44.10it/s]


(4308, 2719)

In [13]:
additional_words = [
    "können", 
    "gebühren",
    "gebühr",
    "schüler", 
    "universitäten", 
    "müssen", 
    "grundsätzlich", 
    "fächern", 
    "maßnahmen", 
    "möglichkeit",
    "studiengänge",
    "qualität",
    "produktivität",
    "prüfungen",
    "bedürfnissen",
    "bedürfen",
    "trägt",
    "förderunterricht",
    "verlängerung",
    "unterstützungsangebot",
    "studienplätze",
    "förderbedarf",
    "zuschüsse",
    "kommunalbehörden",
    "behörden",
    "ausbildungsmaßnahmen",
    "rehabilitationsmaßnahmen",
    "regulären",
    "arbeitsmarktbehörden",
    "behindertenwerkstätten",
    "tätigkeit",
    "während",
    "schulungübernimmt",
    "beschäftigung",
    "höhe",
    "völlig",
    "seriös",
    "vergnügen",
    "millionäre",
    "grosszügigen",
    "benötigt",
    "verfügbar",
    "tätige",
    "dürfen",
    "gängigen",
    "hören",
    "großartige",
    "SpaßMobiles",
    "reguläre",
    "gültig",
    "beträgt",
    "füllen",
    "fülle",
    "großbritannien",
    "kämpfen",
    "nervös",
    "nähe",
    "silberküste",
    "erklären",
    "debütalbum",
    "Livequalitäten",
    "trüffel",
    "weiß",
    "südostküste",
    "günstigen",
    "motorräder",
    "künftig",
    "hosenbügler",
    "frühstücksbuffet",
    "große",
    "großes",
    "erklärt",
    "geäußerten",
    "undäberprüfung",
    "präzisionsprodukte",
    "gießvorgang",
    "gleichmäßige",
    "gießform",
    "präzise",
    "kürzeste",
    "mängel",
    "zumäbermäßigen",
    "statusgemäße",
    "prüfer",
    "patentansprüche",
    "fällt",
    "unstatutengemäße",
    "neuerlicheäberprüfung",
    "könne",
    "lösung",
    "unabhängig",
    "mögen",
    "sekundärer",
    "abschlieäenderöffnung",
    "phänomene",
    "hütte",
    "binär",
    "binäre",
    "heißt",
    "vollständig",
    "enthält",
    "verständnis",
    "jüngeren",
    "verkantete",
    "gültigen",
    "könnte",
    "lagen",
    "präzedenzfälle",
    "bekräftigte",
    "lösende",
    "lösenden",
    "nehmenäwurden",
    "nehmenäwurde",
    "lektüre",
    "autoritäts",
    "dafür",
    "auslösen",
    "lösen",
    "gummisöffnen",
    "räumt",
    "räumte",
    "enthält",
    "nötigen",
    'nötige',
    "gelöst",
    "gelöste",
    "müsste",
    "müssten",
    "erklärte",
    "erklärten",
    "naturphänomen",
    "aktivität",
    "endgültigen",
    "trophäe",
    "popularität",
    "regelmässig",
    "märz",
    "seriöse",
    "mittelständler",
    "kostengünstig",
    "kostengünstige",
    "spüren",
    "luxuriöses",
    "verstoßen",
    "könnens",
    "männer",
    "glücksmail",
    "brüllt",
    "stürmt",
    "prüfen",
    "regelmässig",
    "getätigt",
    "Vergnügungslokalen",
    "rätsel",
    "einemäusserst",
    "stärken",
    "stärke",
    "glastüren",
    "gäste",
    "atmosphäre",
    "gläser",
    "geprüft",
    "münzen",
    "münze",
    "privatsphäre",
    "tür",
    "luxuriöses",
    "luxuriöseste",
    "luxuriösesten",
    "geschäftsmännern",
    "garten",
    "jörg",
    "Vogel",
    "königspalast",
    'gemälden',
    "antiquitäten",
    "gebäuden",
    "könnten",
    "frühstück",
    "Inquisicion",
    "verknüpft",
    "türkisch"
]
resolved.extend(additional_words)
resolved = list(set(resolved))

In [14]:
def replace_unk_words(strings, resolved, separator="�"):
    replaced = []
    pattern = r'[a-zA-Z]+\s+' + re.escape(separator) + r'\s+[a-zA-Z]+'
    for string in tqdm(strings, desc="Extracting words", total=len(strings), ncols=100):
        matches = re.findall(pattern, string)
        for match in matches:
            for resolved_word in resolved:
                if distance(match.replace(" ", "").lower(), resolved_word) == 1:
                    string = string.replace(match, resolved_word)
        replaced.append(string)
    return replaced

In [15]:
de_replaced = replace_unk_words(de_clean, resolved)

Extracting words: 100%|████████████████████████████████| 4468840/4468840 [03:01<00:00, 24603.64it/s]


In [16]:
save_file(de_replaced, "data/clean/clean.de") # 9600 -> 8900 -> 8100

In [14]:
de_clean = read_file("data/clean/clean_v1.de")

In [15]:
# some non German sentences in .de file
# 0 -> 50
# 1345 -> 1360
# 3610 -> 3625

# 308488 -> 308700
# 312085 -> 312180
# 1557748 -> 1557880

def final_clean(strings):
    transformed_strings = []
    for string in tqdm(strings, desc="Cleaning strings", total=len(strings), ncols=100):
        string = re.sub(r', . ', '', string)
        string = re.sub(r'. , ', '', string)
        string = re.sub(r'�', '', string)
        string = re.sub(r'ã ¼ ', 'ü', string)
        string = re.sub(r'ã  ', 'ß', string)
        string = re.sub(r'ã ¶ ', 'ö', string)
        string = re.sub(r'ã ¤ ', 'ä', string)
        string = re.sub(r'ă ¤ ', 'ä', string)
        string = re.sub(r'', '', string)
        transformed_strings.append(string)
    return transformed_strings

In [16]:
de_clean = final_clean(de_clean)
save_file(de_clean, "data/clean/clean_v1.de")

Cleaning strings: 100%|███████████████████████████████| 4468840/4468840 [00:40<00:00, 110731.13it/s]


In [29]:
def fix_apostrophe_en(strings):
    transformed_strings = []
    pattern = r"\b(wasn � t|musn � t|isn � t|can � t|don � t|didn � t|won � t|haven � t|that � s|it � s)\b"
    for string in tqdm(strings, desc="Cleaning strings", total=len(strings), ncols=100):
        string = string.lower()
        string = re.sub(pattern, lambda match: match.group().replace('�', "'"), string)
        transformed_strings.append(string)
    return transformed_strings

In [30]:
de_clean = fix_apostrophe_en(de_clean)

Cleaning strings: 100%|███████████████████████████████| 4468840/4468840 [00:28<00:00, 158735.84it/s]


In [31]:
save_file(de_clean, "data/clean/clean.en")

In [10]:
txt = "ďîääĺđćęŕ číäčâčäóŕëüíűő řŕáëîíîîňâĺ ÷ ŕţůčő çŕ âíĺříčé âčä ńŕéňŕ ."
doc = nlp_model(txt)
language = doc._.language
print(language)

{'language': 'cs', 'score': 0.857137658806668}


In [15]:
de_clean = read_file("data/clean/clean.en")
save_file(de_clean, "data/clean/clean.en")